In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [2]:
model_save_path = '/private/wuchenpeng/projects/high-molecular/models/'

In [3]:
test_data = pd.read_excel('/private/wuchenpeng/projects/high-molecular/data/new_data_01norm.xlsx')

In [4]:
test_data

,redox,azo,fe,ybc,温度,浓度,AMPS,combined_feature,combined_feature1,combined_feature2,[n],转化率（%）
0,0.000000,0.350277,1.000000,0.0,0.000000,0.75,0.2,0.265871,0.000000,0.267841,0.404101,0.818182
1,1.000000,0.000000,0.000000,1.0,1.000000,0.11,0.5,0.806670,1.000000,0.805366,0.000000,0.863636
2,0.278729,1.000000,0.227273,1.0,0.181818,1.00,1.0,0.000000,0.451928,0.000000,1.000000,0.000000
3,1.000000,0.361265,0.000000,1.0,0.763636,0.00,0.0,0.898966,0.900846,0.898989,0.337153,0.818182
4,1.000000,0.315170,0.000000,1.0,0.818182,0.00,0.0,0.932668,0.913497,0.932659,0.376805,0.863636
5,1.000000,0.283293,0.000000,1.0,0.872727,0.00,0.0,0.966341,0.922247,0.966330,0.423102,0.863636
6,1.000000,0.258900,0.000000,1.0,0.927273,0.00,0.0,1.000000,0.928942,1.000000,0.389940,0.909091
7,1.000000,0.361265,0.000000,1.0,0.818182,0.15,0.0,0.921406,0.900846,0.921462,0.433014,0.818182
8,1.000000,0.315170,0.000000,1.0,0.763636,0.15,0.0,0.887886,0.913497,0.887792,0.460989,0.863636
9,1.000000,0.283293,0.000000,1.0,0.927273,0.15,0.0,0.988781,0.922247,0.988803,0.440642,0.863636


In [5]:
X_test = test_data.iloc[:, :-5]
y_test = test_data.iloc[:, -1]

In [6]:
X_test

,redox,azo,fe,ybc,温度,浓度,AMPS
0,0.000000,0.350277,1.000000,0.0,0.000000,0.75,0.2
1,1.000000,0.000000,0.000000,1.0,1.000000,0.11,0.5
2,0.278729,1.000000,0.227273,1.0,0.181818,1.00,1.0
3,1.000000,0.361265,0.000000,1.0,0.763636,0.00,0.0
4,1.000000,0.315170,0.000000,1.0,0.818182,0.00,0.0
5,1.000000,0.283293,0.000000,1.0,0.872727,0.00,0.0
6,1.000000,0.258900,0.000000,1.0,0.927273,0.00,0.0
7,1.000000,0.361265,0.000000,1.0,0.818182,0.15,0.0
8,1.000000,0.315170,0.000000,1.0,0.763636,0.15,0.0
9,1.000000,0.283293,0.000000,1.0,0.927273,0.15,0.0


In [7]:
y_test

0     0.818182
1     0.863636
2     0.000000
3     0.818182
4     0.863636
5     0.863636
6     0.909091
7     0.818182
8     0.863636
9     0.863636
10    0.818182
11    0.863636
12    0.863636
13    0.909091
14    0.954545
15    0.909091
16    0.954545
17    0.863636
18    1.000000
Name: 转化率（%）, dtype: float64

In [12]:
# 特征缩放
scaler2 = StandardScaler()
X_test_scaled = scaler2.transform(X_test)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [10]:
# 加载模型
loaded_model = joblib.load('/private/wuchenpeng/projects/high-molecular/models/Random Forest_zhuanhualv_model.pkl')

In [11]:
y_pred_loaded = loaded_model.predict(X_test)
mse_loaded = mean_squared_error(y_test, y_pred_loaded)
r2_loaded = r2_score(y_test, y_pred_loaded)
print('Loaded Random Forest model performance (MSE, R2):', mse_loaded, r2_loaded)

Loaded Random Forest model performance (MSE, R2): 0.04584024979373639 -0.1208216911504052


/private/wuchenpeng/anaconda3/envs/DL/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
